In [ ]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import joblib
import os
from xgboost import XGBRegressor

# Initialize the main window
root = tk.Tk()
root.title("Prediction of the load of FRP confined elliptical CFST columns under eccentric and concentric loading")

# Create the main canvas
canvas = tk.Canvas(root, width=1100, height=700, background='#FFFFFF')
canvas.pack()

# Specify the image path
image_path = r"C:\Users\asus1\Desktop\ECFSTC.jpg"  # Use a raw string for the file path

# Load the image from the specified path and create an image on the canvas using PIL
try:
    image = Image.open(image_path)
    root.ECFSTC_image = ImageTk.PhotoImage(image)  # Store the image in an attribute of root to ensure it's not garbage collected
    canvas.create_image(600, 10, image=root.ECFSTC_image, anchor="n")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load image: {e}")

# Function to create labels on the canvas
def create_label(text, font, fg, bg, x, y):
    label = tk.Label(root, text=text, font=font, fg=fg, bg=bg)
    canvas.create_window(x, y, anchor="w", window=label)
    return label

# Check if the XGBoost model file exists
model_path_xgb = os.path.expanduser("~/Desktop/XGB1.joblib")
if not os.path.exists(model_path_xgb):
    messagebox.showerror("Error", "XGBoost model file not found.")
    raise SystemExit  # Exit if the model file is not found

# Load the XGBoost model
model_xgb = joblib.load(model_path_xgb)

# Define the GUI elements
label_inputdefinetitle = create_label('Definition of Parameters', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', '#FFFFFF', 20, 460)

param_labels = [
    'X1=Total thickness of layers of FRP wraps multiplied by the Elastic Modulus of FRP',
    'X2=Area of the steel tube multiplied by its yield strength',
    'X3=Area of the concrete core multiplied by the strength of unconfined concrete',
    'X4=Eccentric loading ratio',
    'X5=Aspect ratio of column cross-section',
]
for i, text in enumerate(param_labels, start=1):
    create_label(text, ('Georgia', 14), '#00008B', '#FFFFFF', 30, 470 + i*30)

label_inputs = create_label('Inputs', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', '#FFFFFF', 800, 470)

entry_fields = ['E·tf (N/mm)', 'As·fys (N)', 'Ac·fco (N)', 'λ', 'ɑ']
entries = {}
for i, field in enumerate(entry_fields, start=1):
    label = create_label(f'{field} ', ('Georgia', 12,'italic'), '#FF0000', '#FFFFFF', 770, 470 + i*30)
    entry = tk.Entry(root)
    canvas.create_window(970, 470 + i*30, window=entry)
    entries[field] = entry

label_output = create_label('Output:', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', '#FFFFFF', 250, 680)
label_result = create_label('Prediction of load-caryying capacity (P)', ('Georgia', 16, 'italic'), '#FF0000', '#FFFFFF', 350, 680)

# Predict button command function
def predict():
    # Collect input data
    input_values = []
    for field in entry_fields:
        try:
            value = float(entries[field].get())
            if value <= 0:
                raise ValueError("Value must be positive.")
            input_values.append(value)
        except ValueError as e:
            messagebox.showerror("Error", f"Invalid input for {field}. Please enter a positive numeric value.")
            return

    input_data = np.array([input_values])
    try:
        prediction = model_xgb.predict(input_data)
        label_result['text'] = f'P = {prediction[0]:.4f} kN'
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during prediction: {e}")

# Predict button
predict_button = tk.Button(root, text='Calculate', font=('Georgia', 12), command=predict, fg='#FFFFFF', bg='green')
canvas.create_window(800, 680, window=predict_button)

# Run the main loop
root.mainloop()